In [1]:
import sys
import pandas as pd
# import yfinance as yf
# import kplot as kp

In [2]:
# kline plot with pyecharts and akshare

# from pyecharts.charts import Kline
from pyecharts.charts import Kline, Bar, Grid
from pyecharts import options as opts
import akshare as ak
import pandas as pd

# from pyecharts.globals import CurrentConfig,NotebookType
# CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from typing import List, Tuple, Dict, Any, Union, Optional

def get_aks_KlineData(symbol, period="daily", adjust="qfq", **kwargs):
    data=ak.stock_zh_a_hist(symbol, period, adjust, **kwargs)
    data['color']=data['涨跌幅'].apply(lambda x:True if x>=0 else False)
    data['MA5']=data['收盘'].rolling(5).mean()
    data['MA20']=data['收盘'].rolling(20).mean()
    data['MA80']=data['收盘'].rolling(80).mean()
    data['MA320']=data['收盘'].rolling(320).mean()
    return data

def trans_aksdat_to_echartdat(df_data: pd.DataFrame) -> Tuple:
    """ """
    outdict={}
    for col in df_data.columns:
        outdict[col]=df_data[col].to_list()
    outdict['kline_data'] = df_data[["开盘", "收盘", "最低", "最高"]].values.tolist()
    return outdict


def draw_kline(
    date, kline_data, yaxis_name="Price", title="Stock K-Line Chart", markline=None
) -> Kline:
    """ """
    kline = (
        Kline()
        .add_xaxis(date)
        .add_yaxis(
            yaxis_name,
            y_axis=kline_data,
            markline_opts=opts.MarkLineOpts(
                data=(
                    [
                        opts.MarkLineItem(type_="max", name="Max"),
                        opts.MarkLineItem(type_="min", name="Min"),
                    ]
                    if markline
                    else None
                ),
            ),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            xaxis_opts=opts.AxisOpts(type_="category", is_scale=True),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                is_scale=True,
            ),
            datazoom_opts=[opts.DataZoomOpts(type_="inside"), opts.DataZoomOpts()],
            tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        )
    )
    return kline


def draw_bar(date, volumn_data,color_data, yaxis_name="Volume", title="Stock Volume Chart") -> Bar:
    bar = (
        Bar().add_xaxis(date).add_yaxis(yaxis_name, y_axis=volumn_data)
    ).set_global_opts(
        xaxis_opts=opts.AxisOpts(type_="category", is_scale=True),
        yaxis_opts=opts.AxisOpts(type_="value", is_scale=True),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="vertical"),
        datazoom_opts=[opts.DataZoomOpts(type_="inside"), opts.DataZoomOpts()],
    )

    return bar


class kchartDict(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def getData(self, symbol, period="daily", adjust="qfq", **kwargs):
        data=get_aks_KlineData(symbol, period, adjust, **kwargs)
        self[symbol] = data
        return self[symbol]

    def plot_aks(self, symbol, period="daily", adjust="qfq", **kwargs):
        if symbol not in self.keys():
            self.getData(symbol, period, adjust, **kwargs)
        data_dict = trans_aksdat_to_echartdat(
            self[symbol]
        )
        kline = draw_kline(
            data_dict['日期'], data_dict['kline_data'], symbol, title="stock " + symbol + " Kline Chart"
        )

        volumn_bar = draw_bar(
            data_dict['日期'],
            data_dict['成交量'],
            data_dict['color']
        )

        grid = (
            Grid(init_opts=opts.InitOpts(width="900px", height="600px"))
            .add(
                kline,
                grid_opts=opts.GridOpts(pos_bottom="30%"),
                is_control_axis_index=True,
            )
            .add(
                volumn_bar,
                grid_opts=opts.GridOpts(pos_top="70%"),
            )
        )
        return grid
    


In [3]:
kchart = kchartDict()

In [4]:
kchart.plot_aks(symbol='000005',period='monthly').render_notebook()

### 选股
- 查询所有股票名与代码

1. 运营状况分析
- 所属行业
- 近10年营业额上升趋势
- 营业额组成分析，近5年主营业务占比百分数，是否有转型趋势
- 近10年利润上升趋势
- 近10年毛利率改善，或一直大于 ***?***
- 近5年资产负债率，与行业平均值对比
- 近5年市盈率，与行业平均值对比

2. 资金指标
- 昨日第一天涨停
- 昨日第一天成交量大于近10日平均值的***倍数***
- 近 ***?*** 日成交量呈上升趋势
- 近 ***?*** 日资金净流入

3. 技术指标
- 穿过均线
- 

4. 类型
- 次新股


In [5]:
# 查询所有股票名与代码


def linear(yd: list, xd: list = None):
    xd = xd if len(yd) == len(xd) else [i for i in range(len(yd))]
    yavr=sum(yd)/len(yd)
    coeff=np.polynomial.polynomial.polyfit(x=xd,y=yd,deg=[0,1])
    # 线性回归
    return coeff

# 运营状况分析
def analyze_stock(symbol, years=10):
    # 所属行业
    industry_info = ak.stock_individual_info_em(symbol)['所属行业']
    
    # 近10年营业额上升趋势
    income_statement = ak.stock_financial_abstract(symbol=symbol)
    revenue = income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入']
    revenue_trend = linear(revenue.values[-years:])
    
    # 营业额组成分析，近5年主营业务占比百分数，是否有转型趋势
    main_industry = ak.stock_zygc_em(symbol)
    
    # 近10年利润上升趋势
    net_profit = income_statement[income_statement['REPORT_TYPE'] == '年报']['净利润']
    profit_trend = linear(net_profit.values[-years:])
    
    # 近10年毛利率改善，或一直大于某值
    gross_profit = income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入'] - income_statement[income_statement['REPORT_TYPE'] == '年报']['营业成本']
    gross_margin = gross_profit / income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入']
    gross_margin_trend = linear(gross_margin.values[-years:])
    
    # 近5年资产负债率，与行业平均值对比
    balance_sheet = ak.stock_balance_sheet_by_report_em(symbol=symbol)
    total_assets = balance_sheet['总资产'].values[-5:]
    total_liabilities = balance_sheet['总负债'].values[-5:]
    debt_to_asset_ratio = total_liabilities / total_assets
    
    # 近5年市盈率，与行业平均值对比
    pe_ratio = ak.stock_a_lg_indicator(symbol)['市盈率TTM'].values[-5:]
    
    return {
        'industry': industry_info,
        'revenue_trend': revenue_trend[1],
        'profit_trend': profit_trend[1],
        'gross_margin_trend': gross_margin_trend[1],
        'debt_to_asset_ratio': debt_to_asset_ratio.mean(),
        'pe_ratio': pe_ratio.mean()
    }

# 资金指标
def analyze_funds(symbol, days=10):
    # 昨日第一天涨停
    daily_data = ak.stock_zh_a_hist(symbol, period="daily", adjust="qfq")
    first_limit_up = daily_data[daily_data['涨跌幅'] >= 9.9].iloc[0]
    
    # 昨日第一天成交量大于近10日平均值的倍数
    avg_volume = daily_data['成交量'].rolling(days).mean().iloc[-1]
    yesterday_volume = daily_data['成交量'].iloc[-1]
    volume_ratio = yesterday_volume / avg_volume
    
    # 近N日成交量呈上升趋势
    volume_trend = linear(daily_data['成交量'].values[-days:])
    
    # 近N日资金净流入
    net_inflow = daily_data['资金净流入'].iloc[-days:].sum()
    
    return {
        'first_limit_up': first_limit_up,
        'volume_ratio': volume_ratio,
        'volume_trend': volume_trend[1],
        'net_inflow': net_inflow
    }

# 技术指标
def analyze_technical(symbol):
    # 穿过均线
    daily_data = ak.stock_zh_a_hist(symbol, period="daily", adjust="qfq")
    ma_5 = daily_data['收盘'].rolling(5).mean()
    ma_20 = daily_data['收盘'].rolling(20).mean()
    cross_ma = (ma_5.iloc[-1] > ma_20.iloc[-1]) and (ma_5.iloc[-2] <= ma_20.iloc[-2])
    
    return {
        'cross_ma': cross_ma
    }

# 类型
def is_new_stock(symbol):
    new_stock_data = ak.stock_new_a_spot_em()
    return symbol in new_stock_data['股票代码'].values


In [6]:
all_stocks = ak.stock_zh_a_spot_em() # 所有股票列表

In [7]:
import pandas_datareader as pdr

In [14]:
pdr.get_data_fred('AAPL', start='2020-01-01', end='2023-10-01')

ConnectionError: HTTPSConnectionPool(host='fred.stlouisfed.org', port=443): Read timed out.

In [8]:
all_stocks.columns

Index(['序号', '代码', '名称', '最新价', '涨跌幅', '涨跌额', '成交量', '成交额', '振幅', '最高', '最低',
       '今开', '昨收', '量比', '换手率', '市盈率-动态', '市净率', '总市值', '流通市值', '涨速', '5分钟涨跌',
       '60日涨跌幅', '年初至今涨跌幅'],
      dtype='object')

In [9]:
new_stock=ak.stock_new_a_spot_em() # 新股列表
new_stock.head(5)

0it [00:00, ?it/s]

,序号,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,振幅,最高,...,换手率,市盈率-动态,市净率,上市日期,总市值,流通市值,涨速,5分钟涨跌,60日涨跌幅,年初至今涨跌幅
0,1,001382,新亚电缆,27.13,10.02,2.47,339975,8.633350e+08,13.38,27.13,...,56.53,82.87,6.91,2025-03-21,11177560000,1631584445,0.00,0.00,266.62,266.62
1,2,001356,富岭股份,18.57,10.01,1.69,426579,7.721410e+08,10.49,18.57,...,35.12,50.03,5.63,2025-01-23,10943672400,2255698364,0.00,0.00,250.38,250.38
2,3,603072,天和磁材,53.24,10.00,4.84,306349,1.546907e+09,12.36,53.24,...,47.32,99.05,7.07,2025-01-03,14070267200,3447057128,0.00,0.00,338.19,338.19
3,4,301658,C首航,43.05,5.90,2.40,279585,1.206635e+09,25.85,48.50,...,72.19,68.58,5.80,2025-04-02,17752577362,1667240787,0.19,-0.12,264.83,264.83
4,5,301501,恒鑫生活,55.30,5.33,2.80,69932,3.816530e+08,7.24,55.68,...,28.92,25.65,2.90,2025-03-19,5640600000,1337396325,0.11,0.04,38.53,38.53


In [10]:
ak.stock_individual_info_em('000001') # 股票基本信息

,item,value
0,股票代码,000001
1,股票简称,平安银行
2,总股本,19405918198.0
3,流通股,19405571850.0
4,总市值,220063112365.320007
5,流通市值,220059184779.0
6,行业,银行
7,上市时间,19910403


In [11]:
stock_zygc_ym_df = ak.stock_zygc_em(symbol="600519") # 股票主营业务收入明细
stock_zygc_ym_df.head(5) # 股票主营业务收入明细

KeyError: 'zygcfx'

In [ ]:
# functions for stock with its symbol code
#

def analyze_operating_conditions(symbol, years=10):
    """Analyze operating conditions of a stock"""
    # Get financial statements
    income_statement = ak.stock_financial_abstract(symbol=symbol)
    balance_sheet = ak.stock_balance_sheet_by_report_em(symbol=symbol)
    
    # Analyze revenue trend
    revenue = income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入']
    revenue_trend = linear(revenue.values[-years:])
    
    # Analyze profit trend
    net_profit = income_statement[income_statement['REPORT_TYPE'] == '年报']['净利润']
    profit_trend = linear(net_profit.values[-years:])
    
    # Calculate gross profit margin
    gross_profit = income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入'] - income_statement[income_statement['REPORT_TYPE'] == '年报']['营业成本']
    gross_margin = gross_profit / income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入']
    gross_margin_trend = linear(gross_margin.values[-years:])
    
    # Calculate debt-to-asset ratio
    total_assets = balance_sheet['总资产'].values[-5:]
    total_liabilities = balance_sheet['总负债'].values[-5:]
    debt_to_asset_ratio = total_liabilities / total_assets
    
    return {
        'revenue_trend': revenue_trend[1],
        'profit_trend': profit_trend[1],
        'gross_margin_trend': gross_margin_trend[1],
        'debt_to_asset_ratio': debt_to_asset_ratio.mean()
    }



def get_stock_info(
    symbol,
):  # 获取单只股的信息
    info = ak.stock_individual_info_em(symbol)
    # info1=ak.stock_individual_basic_info_xq(symbol)
    return (info,)  # info1


def get_income(symbol, years):
    data = ak.stock_yjbb_em(date="20220331")  # 年报季报业绩报表
    data = ak.stock_yjyg_em(date="20190331")  # 业绩预告
    return


def cc():
    ak.stock_zcfz_em(date="20240331")  # 资产负债表
    return


def get_main_industry(symbol, years):
    data = ak.stock_zygc_em(symbol)
    return data


def get_profile(symbol, years):
    return


def get_profile_lv(symbol, years):  # 毛利率
    return


def get_(symbol, years):  # 资产负债率
    return


def get_(symbol, years):  # 市盈率
    return


def get_top_days(symbol) -> int:  # 涨停天数
    return


def get_(symbol, days=11, today="today"):  # 当日、10日成交量平均值
    return





def get_newStock():
    data = ak.stock_new_a_spot_em()
    return data

In [ ]:
symbol='SH600518'
years=10

income_statement = ak.stock_financial_abstract(symbol=symbol)
balance_sheet = ak.stock_balance_sheet_by_report_em(symbol=symbol)



  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
income_statement.head(5)


,选项,指标,20240930,20240630,20240331,20231231,20230930,20230630,20230331,20221231,...,20021231,20020930,20020630,20020331,20011231,20010630,20001231,19991231,19981231,19971231
0,常用指标,归母净利润,5.015538e+06,1.576033e+07,7.471308e+06,1.025212e+08,-1.480091e+08,-1.258582e+08,-5.040857e+07,-2.688159e+09,...,4.383265e+07,3.649631e+07,2.608356e+07,8064390.12,2.915688e+07,1.555961e+07,2.455268e+07,2.017724e+07,1.267343e+07,7919781.15
1,常用指标,营业总收入,3.873558e+09,2.474339e+09,1.201240e+09,4.874016e+09,3.534121e+09,2.377002e+09,1.143946e+09,4.180150e+09,...,4.109803e+08,3.005438e+08,1.992158e+08,96254633.55,3.808004e+08,1.910479e+08,3.524015e+08,3.287797e+08,1.881679e+08,81942015.72
2,常用指标,营业成本,4.077730e+09,2.594555e+09,1.270758e+09,4.894618e+09,3.835872e+09,2.581213e+09,1.230355e+09,4.889788e+09,...,3.479043e+08,2.580081e+08,1.722232e+08,84288103.34,3.369447e+08,1.707910e+08,3.134063e+08,3.007014e+08,1.726678e+08,72363444.02
3,常用指标,净利润,8.110876e+06,1.812019e+07,8.612817e+06,1.035669e+08,-1.465388e+08,-1.250650e+08,-4.976538e+07,-2.693800e+09,...,4.383265e+07,3.649631e+07,2.608356e+07,8064390.12,2.915688e+07,1.555961e+07,2.455268e+07,2.535097e+07,1.366780e+07,8570217.52
4,常用指标,扣非净利润,-1.795222e+08,-1.081346e+08,-5.847647e+07,-7.528586e+08,-3.138531e+08,-2.061241e+08,-8.890151e+07,-1.508035e+09,...,4.184426e+07,2.849890e+07,1.808505e+07,8064390.12,2.925024e+07,1.357210e+07,2.458862e+07,2.017724e+07,1.267343e+07,7919781.15


In [ ]:

# Analyze revenue trend
revenue = income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入']
revenue_trend = linear(revenue.values[-years:])

# Analyze profit trend
net_profit = income_statement[income_statement['REPORT_TYPE'] == '年报']['净利润']
profit_trend = linear(net_profit.values[-years:])

# Calculate gross profit margin
gross_profit = income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入'] - income_statement[income_statement['REPORT_TYPE'] == '年报']['营业成本']
gross_margin = gross_profit / income_statement[income_statement['REPORT_TYPE'] == '年报']['营业总收入']
gross_margin_trend = linear(gross_margin.values[-years:])

# Calculate debt-to-asset ratio
total_assets = balance_sheet['总资产'].values[-5:]
total_liabilities = balance_sheet['总负债'].values[-5:]
debt_to_asset_ratio = total_liabilities / total_assets